In [1]:
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [2]:
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  # nrows=100000
                  )

In [3]:
# set column names
colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
ce5.columns = colnames

# Parse 'END' column with optional milliseconds
# ce5['END'] = pd.to_datetime(ce5['END'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce') # don't like coercing errors
# ce5['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce5['END']] # works, but not as efficient

from carb_funcs import optional_ms
# Apply the parsing function to the 'END' column
ce5['END'] = optional_ms(ce5['END'])

# Clean and convert other columns
ce5['TIME'] = ce5['TIME'].str.replace('time', '', regex=False).astype(float)
ce5['USER'] = ce5['USER'].str.replace('user', '', regex=False).astype(int)
ce5['RETRY'] = ce5['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce5['RETURNCODE'] = ce5['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce5['COMMAND'] = ce5['COMMAND'].str.replace('command', '', regex=False)

# Display the updated DataFrame
ce5.head()

,END,USER,RETRY,TIME,RETURNCODE,COMMAND
0,2020-10-16 08:15:39.278699,0,0,0.073476,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
1,2020-10-16 08:18:08.313309,0,0,0.183632,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
2,2020-10-16 08:22:48.128689,0,0,0.075471,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
3,2020-10-16 08:25:13.257408,0,0,0.094844,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."
4,2020-10-16 08:31:01.460723,0,0,0.074988,0.0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '2..."


In [10]:
print(ce5['END'].sort_values().iloc[-1])

2021-10-07 21:59:17.693458


In [5]:
# ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
#                   header=None,
#                   delimiter=' - ',
#                   engine='python',
#                   nrows=100)

In [6]:
# ce6.columns=colnames
# ce6['TIME'] = [float(re.sub(r'time', '', row)) for row in ce6['TIME']]
# ce6['USER'] = [float(re.sub(r'user', '', row)) for row in ce6['USER']]
# ce6['RETRY'] = [float(re.sub(r'retry', '', row)) for row in ce6['RETRY']]
# ce6['RETURNCODE'] = [float(re.sub(r'returncode', '', row)) for row in ce6['RETURNCODE']]
# ce6['COMMAND'] = [re.sub(r'command', '', row) for row in ce6['COMMAND']]
# ce6.head()